In [ ]:
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show
output_notebook()

In [ ]:
from app.utils.process_gtimelog import get_work_df, add_processed_columns
processed = add_processed_columns(get_work_df())
processed.tail()

In [ ]:
import datetime
today = datetime.datetime.today()
today

In [ ]:
one_week_ago = today - datetime.timedelta(weeks=1)
two_week_ago = today - datetime.timedelta(weeks=2)
three_week_ago = today - datetime.timedelta(weeks=3)

start = two_week_ago + datetime.timedelta(days=1)
end = one_week_ago

one_week = processed[(processed.timestamp.dt.date >= start.date()) & (processed.timestamp.dt.date <= end.date())]
one_week = one_week[one_week.activity != 'start']
one_week['formatted_activity'] = one_week.parent_activity + ' (' + one_week.sub_activity + ')'
one_week['activity_bottom'] = one_week.formatted_activity + ':0.25'
one_week['activity_top'] = one_week.formatted_activity + ':0.75'


one_week.tail()

In [ ]:
grouped = one_week.groupby([one_week.timestamp.dt.date, one_week.parent_activity, one_week.formatted_activity]).sum().unstack(0).fillna(0)
grouped

In [ ]:
grouped = one_week.groupby([one_week.timestamp.dt.date, one_week.parent_activity, one_week.formatted_activity]).sum().fillna(0)

#levels = grouped.columns.levels
#labels = grouped.columns.labels
#grouped.columns = levels[1][labels[1]]
#grouped.index.names=['']
grouped

In [ ]:
def make_df(group_by):
    df = one_week.groupby([one_week.timestamp.dt.date, group_by]).sum().unstack(1)
    df.columns = df.columns.levels[1][df.columns.labels[1]]
    df.columns.name = None
    df.index = pd.DatetimeIndex(df.index)
    all_date_index = pd.DatetimeIndex(start=start.date(), end=end.date(), freq='d')
    df = df.join(pd.DataFrame(index=all_date_index), how='outer').fillna(0)
    df.index = df.index.format(formatter=lambda x: x.strftime("%a %b %d"))
    df = df.T
    df['Total'] = df.sum(axis=1)
    return df
parent = make_df(one_week.parent_activity)
parent

In [ ]:
individual = make_df(one_week.formatted_activity)
individual = individual.reset_index()

def get_proportion_of_parent(r):
    activity_total = r['Total']
    activity = r['index']
    parent_total = parent['Total'][activity.split(' ')[0]]
    return '%i%%' % ((( activity_total / parent_total ) * 100) )
    
individual['%'] = individual.apply(get_proportion_of_parent, axis=1)
individual = individual.set_index('index')
individual.index.name = None
individual

In [ ]:
list(individual.columns)

In [ ]:
pd.concat([parent, individual])[list(individual.columns)].fillna('')

In [ ]:
from IPython.display import display, HTML
display(HTML(individual.to_html(header=False)))